In [34]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()

ws = Workspace.get(name='quick-starts-ws-136103',
                   subscription_id='6971f5ac-8af1-446e-8034-05acea24681f',
                   resource_group='aml-quickstarts-136103',
                   )

exp = Experiment(ws, 'myexperiment')

run = exp.start_logging()


In [35]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TOO Create compute cluster
cpu_cluster_name = "cpu-cluster"

# verify that the cluster is not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('cluster exist')
except ComputeTargetException:
    compute_gg = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_gg)

cpu_cluster.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)


cluster exist
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [36]:
from azureml.train.hyperdrive import normal, uniform, choice
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.train.sklearn import SKLearn
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
      '--C': choice( 0.25, 0.5, 0.75, 1.0, 2),
      '--max_iter': choice(100, 150, 200, 250, 300)
  }
)





In [37]:
# Specify policy
policy = BanditPolicy(
           slack_factor = 0.1,
           evaluation_interval = 2,
           delay_evaluation = 5)



In [42]:
if "training" not in os.listdir():
    os.mkdir("./training")

# Creat a SKLearn estimator for the use with train.py
est = SKLearn(source_directory = './',
                     entry_script = './training/train.py',
                     compute_target = cpu_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy
hyperdrive_config = HyperDriveConfig(estimator = est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20,
                             max_concurrent_runs=4)




In [43]:
# Submit your hyperparameter run to the experiment
hyperdrive_run = experiment.submit(config=hyperdrive_config)


In [44]:
# show run details with the widgets
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_59c527d7-f26e-4d14-9709-8c682b5a8dbb
Web View: https://ml.azure.com/experiments/myexperiment/runs/HD_59c527d7-f26e-4d14-9709-8c682b5a8dbb?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-136103/workspaces/quick-starts-ws-136103

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-26T12:31:55.642018][API][INFO]Experiment created<END>\n"<START>[2021-01-26T12:31:56.9523600Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_59c527d7-f26e-4d14-9709-8c682b5a8dbb
Web View: https://ml.azure.com/experiments/myexperiment/runs/HD_59c527d7-f26e-4d14-9709-8c682b5a8dbb?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-136103/workspaces/quick-starts-ws-136103

Warnings:
{
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "User errors were found in at least one of the child runs.",
    "messageF

{'runId': 'HD_59c527d7-f26e-4d14-9709-8c682b5a8dbb',
 'target': 'cpu-cluster',
 'status': 'Canceled',
 'startTimeUtc': '2021-01-26T12:31:55.355358Z',
 'endTimeUtc': '2021-01-26T12:40:24.11188Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name": "Acc

In [45]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [ ]:
import joblib
# Get your best run and save the model from that run.
print(best_run)
best_run_metrics=best_run.get_metrics()


In [ ]:
print("BEST RUN ID AND BEST RUN ACCURACY")
print("Best_run_id",best_run.id)
print("Best_run_accuracy",best_run_metrics['Accuracy'